<a href="https://colab.research.google.com/github/chinge55/sports-news/blob/master/Nepali_Dataset_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import torch
from torch.utils.data import Dataset 
import random
import time
import datetime
import random
from transformers import GPT2LMHeadModel, GPT2Config
import numpy as np
from torch.utils.data import random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
!wget https://raw.githubusercontent.com/sharad461/english-corpus-nepal/master/annapurnaexpress-15k.en

--2021-06-22 07:03:28--  https://raw.githubusercontent.com/sharad461/english-corpus-nepal/master/annapurnaexpress-15k.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1713528 (1.6M) [text/plain]
Saving to: ‘annapurnaexpress-15k.en’

annapurnaexpress-15 100%[===================>]   1.63M  --.-KB/s    in 0.08s   

2021-06-22 07:03:29 (21.4 MB/s) - ‘annapurnaexpress-15k.en’ saved [1713528/1713528]



In [ ]:
#/content/annapurnaexpress-15k.en
data = open("/content/annapurnaexpress-15k.en", 'r')

In [ ]:
sentences = data.read()
sentences

'He used to ask them whether they wanted to become a minister or to go to jail.\nMembers from the two sides APEX approached for this story said they still await an appointment with Indian Prime Minister Modi to submit the report.\nThree other foreigners have mentioned their address resembling Nepal’s and sheltered their money in Swiss banks.\nBut the government does not appear serious about it.\nI do not think so.\nThird, the concepts of inclusion and women’s empowerment were not firmly established and there was no pressure group to take up the issue of women’s representation.\nUnlike other appointments which were endorsed by government secre\xadtaries, Tribhuwan himself had signed off on the devolution of authority to Narayan, who was referred to as ‘the royal secre\xadtary’ as well as ‘the legitimate advisor’.\nThe endless stream of pleasure trips of our bureaucrats to the Middle Kingdom only reinforces this perception.\nIndia had recently won freedom from the British, which had esta

In [ ]:
all_sentences = sentences.split("\n")
all_sentences[:10]

['He used to ask them whether they wanted to become a minister or to go to jail.',
 'Members from the two sides APEX approached for this story said they still await an appointment with Indian Prime Minister Modi to submit the report.',
 'Three other foreigners have mentioned their address resembling Nepal’s and sheltered their money in Swiss banks.',
 'But the government does not appear serious about it.',
 'I do not think so.',
 'Third, the concepts of inclusion and women’s empowerment were not firmly established and there was no pressure group to take up the issue of women’s representation.',
 'Unlike other appointments which were endorsed by government secre\xadtaries, Tribhuwan himself had signed off on the devolution of authority to Narayan, who was referred to as ‘the royal secre\xadtary’ as well as ‘the legitimate advisor’.',
 'The endless stream of pleasure trips of our bureaucrats to the Middle Kingdom only reinforces this perception.',
 'India had recently won freedom from th

In [ ]:
len(all_sentences)

15264

In [ ]:
from transformers import GPT2Tokenizer

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<sos>', eos_token='<eos>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(tokenizer.encode('Nepal'))
print(tokenizer.encode('Nepal is a country'))
print(tokenizer.encode('a country'))

[45, 538, 282]
[45, 538, 282, 318, 257, 1499]
[64, 1499]


In [ ]:
max_len = max([len(tokenizer.encode(s)) for s in all_sentences])

print(f"max_len {max_len}")

max_len 143


In [ ]:
def tokenize_seq(sent,tokenizer,max_length):
  return tokenizer('<sos>'+ sent + '<eos>', truncation=True, max_length=max_length, padding="max_length")


In [ ]:
class SportsDataset(Dataset):
  def __init__(self, sentences, tokenizer, gpt2_type = "gpt2", max_length = max_len):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for sentence in sentences:
      encodings = tokenize_seq(sentence, tokenizer, max_length)

      self.input_ids.append(torch.tensor(encodings['input_ids']))
      self.attn_masks.append(torch.tensor(encodings['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]
def format_time(elapsed):
  return str(datetime.timedelta(seconds = int(round(elapsed))))

In [ ]:
dataset = SportsDataset(all_sentences, tokenizer, max_length = max_len)

train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

In [ ]:
train_set, val_set = random_split(dataset, [train_size, val_size])

In [ ]:
dataset[0]

(tensor([50257,  1544,   973,   284,  1265,   606,  1771,   484,  2227,   284,
          1716,   257,  5342,   393,   284,   467,   284,  7356,    13, 50258,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
         50259, 50259, 50259, 50259, 50259, 50259, 5

In [ ]:
train_dataloader = DataLoader(train_set,  sampler = RandomSampler(train_set), batch_size = 16)


In [ ]:
validation_dataloader = DataLoader(val_set, sampler = SequentialSampler(val_set), batch_size = 16)

In [ ]:
# Create default config
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
# Load pretrained gpt2
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)
model.resize_token_embeddings(len(tokenizer))

# Create device
device = torch.device("cuda")
model.cuda()


optimizer = torch.optim.Adam(model.parameters(),lr = 0.0005)
model = model.to(device)

In [ ]:
def eval_keywords(keywords):
  model.eval()
  for keyword in keywords:
    input_seq = "<sos>" + keyword
    generated = torch.tensor(tokenizer.encode(input_seq)).unsqueeze(0)
    generated = generated.to(device)
    sample_outputs = model.generate(
        generated, 
        do_sample = True,
        top_k = 30,
        max_length = 50,
        top_p = 0.90,
        num_return_sequences = 2
    )
    for i, sample_output in enumerate(sample_outputs):
      print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

keywords = ["Nepal", "Sports", "Football", "Paras Khadka"]

In [ ]:
def process_one_batch(batch):
  b_input_ids = batch[0].to(device)
  b_labels = batch[0].to(device)
  b_masks = batch[1].to(device)
  outputs = model(b_input_ids, attention_mask = b_masks, labels = b_labels)
  return outputs

In [ ]:
def train_epoch():
  t0 = time.time()
  total_train_loss = 0
  model.train()
  for step, batch in enumerate(train_dataloader):

    model.zero_grad()
    outputs = process_one_batch(batch)
    loss = outputs[0]
    batch_loss = loss.item()
    total_train_loss += batch_loss

    loss.backward()
    optimizer.step()
      
  avg_train_loss = total_train_loss / len(train_dataloader)  
  print("avg_train_loss",avg_train_loss)  
  elapsed_time = format_time(time.time() - t0)
  print("elapsed time for 1 training epoch : ",elapsed_time)
  
  

In [ ]:
def eval_epoch():
  t0 = time.time()
  total_eval_loss = 0
  nb_eval_steps = 0

  for batch in validation_dataloader:
    with torch.no_grad():
      outputs = process_one_batch(batch)
      loss = outputs[0]
      batch_loss = loss.item()
      total_eval_loss += batch_loss
      
  avg_val_loss = total_eval_loss / len(validation_dataloader)
  print("avg_val_loss",avg_val_loss) 
  elapsed_time = format_time(time.time() - t0)
  print("elapsed time for 1 eval epoch : ",elapsed_time)    

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 3            |        cudaMalloc retries: 3         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   10770 MB |   10770 MB |   43648 MB |   32877 MB |\n|       from large pool |   10756 MB |   10756 MB |   43617 MB |   32861 MB |\n|       from small pool |      14 MB |      14 MB |      30 MB |      16 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |   10770 MB |   10770 MB |   43648 MB |   32877 MB |\n|       from large pool |   10756 MB |   10756 MB |

In [ ]:
train_epoch()

avg_train_loss 1.0643348232067784
elapsed time for 1 training epoch :  0:17:30


In [ ]:
eval_epoch()

avg_val_loss 0.702031639094154
elapsed time for 1 eval epoch :  0:00:40


In [ ]:
eval_keywords(keywords)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Nepal has been raising the price of alcohol to Rs 6,000 per kg, which, in comparison, has raised the price of tobacco and cigarettes by Rs 60,000.
1: Nepal’s government has no interest in supporting the BRI.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Sports fans are increasingly involved in the local elections.
1: Sports are no more than a sport.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Football, music, dance, and more music will not be allowed to be heard in the stadium, but music will be banned from the premises.
1: Football, tennis, boxing, boxing, music, and dance all serve a special purpose for the athletes.
0: Paras Khadka, a 24-year-old from Khokharpur, says, “People of Kathmandu are waiting for a visit to Kathmandu.”
1: Paras Khadka and her brother-in-law, Gyanendra Koirala, are among the party leadership.


In [ ]:
for i in range(3):
  train_epoch()
  eval(epoch)
  eval_keywords(keywords)

KeyboardInterrupt: ignored